### make corpus

In [3]:
from dbConn.mongo_conn import config

conn = config()
col = conn['travel_ai'].blog_contents_ner

cleaned_content = col.find({'cleaned_content': {'$exists':True, '$ne': []}}, {'cleaned_content': 1})

In [4]:
%%time
input_f = "./data/preprocessed_corpus.txt"
f = open(input_f, "w")
for cont in cleaned_content:
    docs = [c for c in cont['cleaned_content'] if c]
    f.write('\n'.join(docs))
del cleaned_content

f.close()
conn.close()

5594986it [02:04, 45070.46it/s]


In [5]:
with open("./data/preprocessed_corpus.txt") as f:
    lines = f.readlines()
len(lines)

65543688

### Tokenizer

In [2]:
train_corpus = "./data/preprocessed_corpus.txt"

In [3]:
from konlpy.tag import Mecab
from huggingface_konlpy.tokenizers_konlpy import KoNLPyBertWordPieceTrainer

mecab_wordpiece_usetag_trainer = KoNLPyBertWordPieceTrainer(
    Mecab(), use_tag=True)
mecab_wordpiece_usetag_trainer.train(
    files=[train_corpus], vocab_size=150000)
mecab_wordpiece_usetag_trainer.save_model('./tokenizer/konlpy_tokenizer')

Train vocab 1/1: 100%|██████████| 60201262/60201262 [2:58:18<00:00, 5627.20it/s]  


[/workspace/travel_ai/tokenizer/konlpy_tokenizer/vocab.txt]


In [4]:
from huggingface_konlpy.transformers_konlpy import KoNLPyBertTokenizer
from huggingface_konlpy.tokenizers_konlpy import KoNLPyWordPieceTokenizer

In [5]:
mecab_bert_usetag = KoNLPyBertTokenizer(
    konlpy_wordpiece = KoNLPyWordPieceTokenizer(Mecab(), use_tag=True),
    vocab_file = './tokenizer/konlpy_tokenizer/vocab.txt'
)

In [8]:
test_sent = "하우스 블렌드 진하고 깔끔하게 균형잡힌맛보헤미안의 대표 시그니처커피이 커피는 뜨거운커피로만 마실 수 있더라구요예쁜 커피잔에 나오니 뭔가 더 고급진 느낌이들어요"
mecab_bert_usetag.tokenize(test_sent)

['하우스/NNP',
 '블/NNG',
 '##렌/NNG',
 '##드/NNG',
 '진하/VA',
 '##고/EC',
 '깔끔/XR',
 '##하/XSA',
 '##게/EC',
 '균형/NNG',
 '##잡힌/VV+ETM',
 '##맛/NNG',
 '##보헤미안/NNG',
 '##의/NNG',
 '대표/NNG',
 '시그/NNP',
 '##니/VCP+EC',
 '##처/NNG',
 '##커피/NNG',
 '##이/VCP',
 '커피/NNG',
 '##는/JX',
 '뜨거운/VA+ETM',
 '##커피/NNG',
 '##로/JKB',
 '##만/JX',
 '마실/NNG',
 '수/NNG',
 '있/VA',
 '##더라구요/EF',
 '##예쁜/VA+ETM',
 '커피/NNG',
 '##잔/NNG',
 '##에/JKB',
 '나오/VV',
 '##니/EC',
 '뭔가/NP+VCP+EC',
 '더/MAG',
 '고급/NNG',
 '##진/XSV+EC+JX',
 '느낌/NNG',
 '##이/JKS',
 '##들/VV',
 '##어요/EF']

In [ ]:
from konlpy.tag import Mecab
m = Mecab()


f = open(input_f, "r")

output_tokens = ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]']
line = None
while line != ' ':
    line = f.readline().strip()
    for l in line.split():
        count = 0
        for token in m.morphs(l):
            tk = token
            if count > 0:
                tk = '##' + tk
                if tk not in output_tokens:
                    output_tokens.append(tk)
            else:
                count += 1
                if tk not in output_tokens:
                    output_tokens.append(tk)

f.close()

In [ ]:
voca_file = open("/mecab_tokenizer/vocab.txt", 'w')

for token in output_tokens:
    voca_file.write(token + '\n')

voca_file.close()

In [ ]:
output_tokens[5:10]